In [1]:
from tensorflow import keras

In [2]:
model = keras.models.load_model('recommendation')

Metal device set to: Apple M1 Pro


In [3]:
model.summary()

Model: "recommender_net_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     multiple                  401216    
                                                                 
 embedding_9 (Embedding)     multiple                  25076     
                                                                 
 embedding_10 (Embedding)    multiple                  2574416   
                                                                 
 embedding_11 (Embedding)    multiple                  160901    
                                                                 
Total params: 3,161,609
Trainable params: 3,161,609
Non-trainable params: 0
_________________________________________________________________


In [9]:
user_embedding = model.user_embedding.get_weights()[0]
user_embedding

array([[-1.2482734e-11, -4.6113757e-10,  3.1781793e-13, ...,
        -8.7456362e-12, -1.6083565e-08,  6.1290621e-13],
       [-4.3149123e-11, -1.4943202e-08, -7.3642532e-09, ...,
        -4.0835452e-08, -7.6951139e-09,  9.3008694e-08],
       [-3.8467115e-07,  8.0289976e-11,  4.7422271e-09, ...,
         2.5930722e-08, -1.3445818e-09,  9.4443453e-10],
       ...,
       [ 5.3582189e-38, -2.4231987e-38, -2.0584793e-38, ...,
        -2.2189303e-38,  6.1500187e-41,  3.4746160e-38],
       [-2.1060462e-38,  4.0677094e-38, -2.5700492e-38, ...,
        -3.2733127e-38,  5.4699865e-38,  4.0983300e-38],
       [ 1.3790607e-38,  4.9799210e-38,  6.6246217e-39, ...,
        -3.4803080e-38,  5.2322725e-38, -4.2692127e-38]], dtype=float32)

In [11]:
recipe_embedding = model.book_embedding.get_weights()[0]
recipe_embedding

array([[-1.5226881e-08, -6.0803904e-07, -1.3793978e-11, ...,
         4.0509622e-07,  1.4212110e-06,  9.1095603e-12],
       [-7.2413052e-08,  1.2832966e-05, -6.6393477e-07, ...,
        -6.5502409e-06,  6.5237191e-07, -4.9938762e-08],
       [-4.9312803e-09, -1.3144530e-07, -3.6259542e-09, ...,
         9.7115276e-08,  1.1160339e-07,  1.3875087e-11],
       ...,
       [ 3.2154151e-11, -3.2611539e-38,  4.5145313e-38, ...,
         5.4980618e-38, -1.4651231e-38,  3.7171247e-38],
       [-5.3037359e-38,  3.8472722e-38,  1.5764159e-39, ...,
        -5.5844137e-38, -4.0527194e-38,  3.8043788e-38],
       [-1.8484399e-38,  2.8322736e-38, -4.0138911e-38, ...,
         4.5773807e-38,  2.3441818e-38, -1.0859095e-38]], dtype=float32)

In [12]:
import pandas as pd

data = pd.read_csv('../dataset/interactions_train.csv')
data.head()

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [14]:
# suggest recipes for user_id 2046
user_id = 2046

In [15]:
import numpy as np

# get the user embedding
index = np.where(data['user_id'] == user_id)[0][0]
index

0

In [16]:
user_embedding[index]

array([-1.24827336e-11, -4.61137573e-10,  3.17817929e-13,  2.40696565e-08,
       -4.49480240e-06,  1.24178055e-11, -3.36472100e-10, -2.37490183e-09,
        3.01764343e-11,  3.15780824e-09,  1.44995960e-10, -6.40612008e-10,
        5.52481971e-09, -8.74563616e-12, -1.60835647e-08,  6.12906210e-13],
      dtype=float32)

In [17]:
# calculate the cosine similarity between the user embedding and all recipe embeddings
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(user_embedding[index].reshape(1, -1), recipe_embedding)

In [18]:
similarity

array([[ 4.3215930e-01,  2.8955659e-02,  6.5072961e-03, ...,
        -8.9294895e-17, -1.3206626e-13, -3.0907588e-38]], dtype=float32)

In [19]:
# get the top 10 most similar recipes
top10 = np.argsort(similarity[0])[-10:]

In [20]:
top10

array([ 81306, 129363,  64120, 109725,  76737,  36311, 135682,  44196,
        14692, 117191])

In [21]:
# get the recipe ids of the top 10 most similar recipes
recipe_ids = data['recipe_id'].unique()[top10]

In [22]:
recipe_ids

array([ 74683, 189308, 173472, 359558, 239160, 130795, 437046,  94696,
        68970, 112137])

In [25]:
# get the recipe names of the top 10 most similar recipes
metadata = pd.read_csv('../dataset/RAW_recipes.csv')

In [26]:
metadata.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [27]:
metadata[metadata['id'].isin(recipe_ids)]['name']

34479                    camping   meal in one packages
40315                     cheesy creamy garlic potatoes
66666                                crock pot chai tea
107112                    honey   spice crinkle cookies
145102                          northwest chicken salad
149412                            orange soymilk frappe
157427                      penne with chorizo and kale
196298                               spicy mustard eggs
226113    white chocolate and peppermint cookie brittle
229953                       yummy garlicky green beans
Name: name, dtype: object

In [32]:
# suggest similar recipes for recipe_id 107112
recipe_id = 4684

In [33]:
# get the recipe embedding
index = np.where(data['recipe_id'] == recipe_id)[0][0]

In [34]:
recipe_embedding[index]

array([-1.5226881e-08, -6.0803904e-07, -1.3793978e-11,  4.2111384e-12,
       -8.6829112e-07,  3.3826217e-07,  6.8160533e-11,  3.3575768e-07,
        3.7193485e-07,  3.0458804e-09,  5.6160104e-07, -2.7985760e-11,
        6.4248162e-09,  4.0509622e-07,  1.4212110e-06,  9.1095603e-12],
      dtype=float32)

In [35]:
# calculate the cosine similarity between the recipe embedding and all recipe embeddings
similarity = cosine_similarity(recipe_embedding[index].reshape(1, -1), recipe_embedding)

In [36]:
similarity

array([[ 9.9999994e-01, -3.4556323e-01,  3.6990246e-01, ...,
        -2.4514863e-13, -2.8669526e-13, -2.3327402e-38]], dtype=float32)

In [37]:
# get the top 10 most similar recipes
top10 = np.argsort(similarity[0])[-10:]

In [38]:
top10

array([114680,  43818,  35800, 108202,  26433, 122827,  12194,  17617,
        67273,      0])

In [39]:
# get the recipe ids of the top 10 most similar recipes
recipe_ids = data['recipe_id'].unique()[top10]

In [40]:
recipe_ids

array([372353, 123414, 127624,  44555, 101771, 185628,  49073,  78347,
       199179,   4684])

In [43]:
# get the recipe names of the top 10 most similar recipes
metadata[metadata['id'].isin(recipe_ids)]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
6826,apple graham cracker dessert,199179,60,22015,2006-12-04,"['60-minutes-or-less', 'time-to-make', 'course...","[141.8, 12.0, 23.0, 15.0, 7.0, 13.0, 4.0]",12,"['preheat oven to 350 degrees', 'spray a 9x9-i...",a new twist - almost a combination of pie and ...,"['light apple pie filling', 'butter', 'splenda...",10
34430,campbells chicken zucchini,78347,45,43083,2003-12-09,"['60-minutes-or-less', 'time-to-make', 'course...","[243.3, 20.0, 9.0, 27.0, 42.0, 27.0, 3.0]",8,"['mix soup , milk , onion , cheese& sour cream...",i got this off the back of a campbells soup ca...,"['cream of mushroom soup', 'milk', 'onion', 'p...",8
35010,cantaloupe red wine surprise,123414,20,67899,2005-05-23,"['lactose', '30-minutes-or-less', 'time-to-mak...","[186.5, 0.0, 84.0, 1.0, 2.0, 0.0, 8.0]",12,['warm wine in a shallow saucepan over low hea...,now this is the way to eat fruit! the recipe i...,"['red wine', 'sugar', 'cantaloupe', 'lemon', '...",5
84749,flank steak with lime chipotle sauce,4684,75,1533,1999-11-18,"['time-to-make', 'course', 'main-ingredient', ...","[487.8, 16.0, 283.0, 13.0, 52.0, 20.0, 26.0]",6,"['mix honey with chilies , mustard , lime juic...",yum,"['flank steaks', 'honey', 'chipotle chiles in ...",10
106419,homemade chicken noodle soup with garlic chili...,101771,60,37449,2004-10-12,"['60-minutes-or-less', 'time-to-make', 'course...","[915.2, 69.0, 66.0, 35.0, 135.0, 58.0, 19.0]",26,"['to make stock , rinse chicken parts in cold ...","chicken soup is not only good for a cold, it m...","['chicken', 'celery', 'carrots', 'onions', 'bl...",18
123298,lentils with mushrooms in a pressure cooker,185628,35,346475,2006-09-11,"['60-minutes-or-less', 'time-to-make', 'course...","[281.9, 16.0, 12.0, 1.0, 27.0, 15.0, 11.0]",9,"['chop the onion , celery , carrot , and mushr...","any mushrooms will do, but i recommend using c...","['celery', 'carrot', 'onion', 'mushrooms', 'ga...",11
153162,parmesan mustard mashed potatoes,49073,38,27416,2002-12-17,"['60-minutes-or-less', 'time-to-make', 'course...","[274.8, 13.0, 18.0, 20.0, 21.0, 26.0, 12.0]",19,"['if using red potatoes , scrub skins off: do ...",a very tasty and easy to make mashed potato re...,"['red potatoes', 'whole milk', 'low-fat sour c...",8
160213,pineapple jam,127624,55,54271,2005-06-27,"['60-minutes-or-less', 'time-to-make', 'course...","[555.7, 0.0, 537.0, 0.0, 0.0, 0.0, 48.0]",17,['add enough water to undrained pineapple to m...,found on kraftfoods.com,"['crushed pineapple in syrup', 'sure-jell', 'b...",4
188963,slow cooker hot crab dip,44555,80,29300,2002-10-29,"['weeknight', 'time-to-make', 'course', 'main-...","[115.0, 14.0, 6.0, 10.0, 11.0, 24.0, 0.0]",9,"['preheat oven to 350 degrees', 'spray a slow ...",NaN,"['butter-flavored cooking spray', 'cream chees...",11
204925,swedish cabbage and orange salad,372353,8,424680,2009-05-17,"['15-minutes-or-less', 'time-to-make', 'course...","[96.9, 5.0, 47.0, 0.0, 2.0, 2.0, 5.0]",3,"['in a covered container , combine cabbage , o...","this recipe was found in the 1973 cookbook, sc...","['cabbage', 'navel oranges', 'golden raisin', ...",5


In [44]:
# get the recipe names of the original recipe
metadata[metadata['id'] == recipe_id]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
84749,flank steak with lime chipotle sauce,4684,75,1533,1999-11-18,"['time-to-make', 'course', 'main-ingredient', ...","[487.8, 16.0, 283.0, 13.0, 52.0, 20.0, 26.0]",6,"['mix honey with chilies , mustard , lime juic...",yum,"['flank steaks', 'honey', 'chipotle chiles in ...",10


In [45]:
recipe_ids = np.arange(len(recipe_embedding))

In [46]:
recipe_ids

array([     0,      1,      2, ..., 160898, 160899, 160900])

In [47]:
# calculate similarity scores for each recipe
similarity_scores = np.dot(user_embedding, recipe_embedding.T)

In [48]:
similarity_scores

array([[ 3.8795607e-12,  2.9956261e-12,  1.1537303e-13, ...,
        -4.0137170e-22, -5.9362477e-19, -1.3872855e-43],
       [-4.0591350e-13, -5.5439849e-13, -7.3628300e-13, ...,
        -1.3874234e-21,  1.9092851e-17,  2.2420775e-44],
       [ 9.9668822e-14, -2.5891294e-12,  2.8330463e-13, ...,
        -1.2368775e-17, -7.8645210e-19,  1.4012985e-45],
       ...,
       [ 6.1657132e-44, -1.1098284e-42, -4.4841551e-44, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 7.7071416e-44,  1.5162049e-42,  2.4382593e-43, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-5.1848043e-44,  1.8286945e-42, -4.0637655e-44, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [49]:
sorted_recipe_ids = np.argsort(similarity_scores)[::-1]

In [50]:
sorted_recipe_ids

array([[ 56166,  90502,  53415, ...,  88185, 132000, 118356],
       [122887, 113047,  88870, ..., 118213,  88185,  77424],
       [ 88185, 139442,  84140, ...,  51304, 105332, 138465],
       ...,
       [130692, 138465,  85469, ...,  74399,  39748,  90043],
       [ 42382, 110823,  93550, ..., 130471,  67521, 132000],
       [132638, 122296,  65128, ...,  39748, 110823,  69053]])

In [51]:
def get_already_interacted_recipe_ids(user_id):
    return data[data['user_id'] == user_id]['recipe_id'].unique()

In [ ]:
# exclude recipes that user has already interacted with
already_interacted_recipe_ids = get_already_interacted_recipe_ids(user_id)
sorted_recipe_ids = np.setdiff1d(sorted_recipe_ids, already_interacted_recipe_ids)

In [ ]:
sorted_recipe_ids

In [ ]:
n = 10
top_n_recipe_ids = sorted_recipe_ids[:n]
top_n_recipe_latent_factors = recipe_embedding[top_n_recipe_ids]

In [ ]:
import numpy as np

# recipe_id and recipe_latent_factors are assumed to be given
recipe_ids = np.arange(len(recipe_embedding))

# calculate similarity scores for each recipe
similarity_scores = np.dot(recipe_embedding, recipe_embedding.T)

# sort recipes by similarity score
sorted_recipe_ids = np.argsort(similarity_scores[recipe_id])[::-1]

# exclude the recipe itself
sorted_recipe_ids = np.setdiff1d(sorted_recipe_ids, recipe_id)

# return top N recommendations
n = 10
top_n_recipe_ids = sorted_recipe_ids[:n]
top_n_recipe_latent_factors = recipe_latent_factors[top_n_recipe_ids]